PRACTICE. Сбор и обработка данных

Важно иметь в виду: все задания необходимо выполнять с использованием данных пользователей, зарегистрировавшихся в 2018 году.

In [521]:
import pandas as pd
import numpy as np

purchase = pd.read_csv(r'C:\Users\Yana\Documents\purchase.csv', sep=',')
events = pd.read_csv(r'C:\Users\Yana\Documents\7_4_Events.csv', sep=',')

In [522]:
events['selected_level'].value_counts()

medium    17408
easy       9011
hard       4667
Name: selected_level, dtype: int64

In [523]:
#events_df.info()
# events_df['start_time'] = pd.to_datetime(events_df['start_time']) --> ERROR, have to separate date on T

In [524]:
# Для начала выведем первые 10 строк датафрейма events_df с помощью команды head(), чтобы ознакомиться с данными.
events.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


In [525]:
# Мы, конечно, можем уже что-то сказать по поводу данных, посмотрев на первые 10 строк, 
# но лучше использовать более точные методы анализа. 
# Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме, 
# для того чтобы оценить, какие данные содержатся в датафрейме events_df.
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


Итак, мы видим, что датафрейм events_df содержит 35808 строк и шесть столбцов. При этом все строки содержат значения в столбцах id, user_id, start_time, event_type, а остальные столбцы содержат пропущенные значения (например, столбец selected_level содержит непустые значения только в 4412 строках).

To retrieve all the rows which startwith required string

dataFrameOut = dataFrame[dataFrame['column name'].str.match('string')]

To retrieve all the rows which contains required string

dataFrameOut = dataFrame[dataFrame['column name'].str.contains('string')]

In [526]:
events.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [527]:
# фильтрацию проводим в два этапа
# выделим пользователей, зарегистрированных в 2018 г.
условие = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[условие]  
lst_reg = registered['user_id'].to_list()
# список пользователей, зарег. в 2018
# второе - оставим в датафрейме только этих пользователей и все их действия уже без привязки к году
events_df = events[events.user_id.isin(lst_reg)]
# Сколько строк содержится в датафрейме events_df? 66959
events_df.shape

(66959, 6)

In [528]:
# Сколько непустых значений содержится в столбце selected_level? 8342 
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


Исследуем пропуски

In [529]:
# Наличие пропущенных строк обусловлено тем, что не все из параметров 
# selected_level и tutorial_id присутствуют в каждом из событий. 
# Эти параметры заполнятся в зависимости от event_type.
# К примеру, посмотрим на events_df, оставив в нём только такие строки, где event_type = level_choice.

events_df[events_df['event_type'] == 'level_choice'].info()

# Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level, 
# но зато содержит пропуски в tutorial_id. Это связано с тем, 
# что для событий типа level_choice не предусмотрена запись параметра tutorial_id.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              8342 non-null   int64  
 1   event_type      8342 non-null   object 
 2   selected_level  8342 non-null   object 
 3   start_time      8342 non-null   object 
 4   tutorial_id     0 non-null      float64
 5   user_id         8342 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 456.2+ KB


In [530]:
# Теперь проверим аналогичные данные, но при условии, 
# что срез будет содержать данные о событиях tutorial_start и tutorial_finish.

events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

# Столбец selected_level в таком срезе не содержит ни одного значения. 
# Причина этого в том, что для событий, связанных с обучением,
# отсутствуют параметры выбора уровня сложности selected_level. 
# А вот остальные столбцы заполнены полностью.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 51412 to 118360
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              32954 non-null  int64  
 1   event_type      32954 non-null  object 
 2   selected_level  0 non-null      object 
 3   start_time      32954 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         32954 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.8+ MB


Исследуем значения

In [531]:
# Вызовем метод describe(), чтобы оценить характеристики каждого столбца. 
# По умолчанию метод describe() выдаёт характеристики 
# только по столбцам с численными типами (например, int64, float64). 
# Мы вызовем этот метод с параметром include='all' для того, чтобы отображать характеристики для всех столбцов.

events_df.describe(include='all')

,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13T09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000
50%,113787.000000,NaN,NaN,NaN,40529.500000,37719.000000
75%,130526.500000,NaN,NaN,NaN,45057.750000,42733.000000


In [532]:
# Давайте оценим, какие уникальные события есть в колонках event_type и selected_level.

display(events_df['event_type'].unique())
events_df['selected_level'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [533]:
# Также оценим, какое количество пользователей совершали события: 19926

events_df['user_id'].nunique()

19926

In [534]:
# Теперь нам нужно проверить, как события, совершённые пользователями, связаны с последующими оплатами пакетов вопросов.
# Выведем первые 10 строк датафрейма purchase_df с помощью команды head().

purchase.head(10)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
5,15679,12989,2016-05-14T02:19:20,200
6,15680,12684,2016-05-14T03:10:14,100
7,15681,12656,2016-05-14T03:42:48,100
8,15682,12624,2016-05-14T06:00:45,50
9,15683,13136,2016-05-14T07:31:03,50


In [535]:
# Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме, 
# для того чтобы оценить, какие данные содержатся в датафрейме purchase_df.

purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [536]:
# Важно! Напоминаем, что все задания необходимо выполнять 
# с использованием данных пользователей, зарегистрировавшихся в 2018 году.
purchase_df = purchase[purchase.user_id.isin(lst_reg)]
# Сколько строк содержится в датафрейме purchase_df? 1600
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


In [537]:
# Снова обратимся к методу describe(), чтобы оценить характеристики каждого столбца датафрейма purchase_df.
purchase_df.describe()
# Какое среднее значение в столбце amount? 110,73

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


Сбор и обработка данных выполнены!

Анализ пользовательских событий

In [538]:
# Изучаем событие registration
# Для начала посмотрим, сколько пользователей совершают событие registration.
# Сколько пользователей совершили событие registration? 19926
display(events_df[events_df['event_type'] == 'registration']['user_id'].nunique())
display(events_df['user_id'].nunique())
# Какой вывод мы можем сделать? Как мы видим, все пользователи совершают событие registration. 
# Это обусловлено тем, что данный этап является обязательным для выполнения и без него пользователь 
# не сможет перейти дальше.

19926

19926

In [539]:
# Изучаем событие tutorial_start 
# Посмотрим на срез данных по событию tutorial_start.
events_df[events_df['event_type'] == 'tutorial_start'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01T14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01T15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01T15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01T17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01T19:11:36,31509.0,27839
51422,80325,tutorial_start,NaN,2018-01-01T19:46:11,31510.0,27834
51434,80337,tutorial_start,NaN,2018-01-02T02:07:07,31511.0,27840
51435,80338,tutorial_start,NaN,2018-01-02T03:03:44,31512.0,27845
51437,80340,tutorial_start,NaN,2018-01-02T04:55:11,31513.0,27842
51446,80349,tutorial_start,NaN,2018-01-02T07:08:00,31514.0,27845


In [540]:
# Посмотрим на количество пользователей, которые совершают событие tutorial_start: 11858

events_df[events_df['event_type'] == 'tutorial_start']['user_id'].nunique()

# Как мы видим, число пользователей, которые перешли к выполнению обучения, меньше, 
# чем число пользователей, прошедших регистрацию.

11858

In [541]:
# Давайте определим процент пользователей, которые перешли к выполнению обучения, 
# и запишем его в переменную percent_tutorial_start_users.

registered_users_count = events_df[events_df["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_df[events_df["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

# Какой процент составляют пользователи, начавшие обучение (от общего числа зарегистрировавшихся)? 59,51

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


In [542]:
# Изучаем событие tutorial_finish
# Теперь давайте посмотрим, какое количество пользователей проходит обучение до конца (событие tutorial_finish).

events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].nunique()

10250

In [543]:
# Рассчитаем процент пользователей, завершивших обучение, среди пользователей, которые начали обучение.
# Это будет показатель tutorial_completion_rate (коэффициент «завершаемости» обучения).

tutorial_finish_users_count = events_df[events_df["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

# Какой процент составляют пользователи, завершившие обучение (от числа пользователей, начавших обучение)? 86.44

Процент пользователей, завершивших обучение: 86.44%


Какой вывод мы можем сделать?

В нашем приложении достаточно хороший процент прохождения обучения. Но есть куда стремиться для его увеличения.

Подумайте, как показатель прохождения может влиять на весь путь пользователя в дальнейшем.

In [544]:
# Изучаем level_choice
# Следующий этап — это выбор уровня сложности level_choice. 
# Давайте посмотрим на процент тех, кто доходит до этого этапа.

events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique()

8342

In [545]:
# Из логики нашего приложения следует, что выбор уровня сложности можно совершить и без прохождения обучения. 
# Поэтому рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень сложности, 
# от общего числа зарегистрировавшихся:

level_choice_users_count = events_df[events_df["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


In [546]:
#  Изучаем pack_choice
# Теперь давайте определим число пользователей, 
# которые совершили событие выбора бесплатного пакета вопросов pack_choice.

events_df[events_df['event_type'] == 'pack_choice']['user_id'].nunique()

5737

In [547]:
# Оценим процент таких пользователей от числа пользователей, которые выбрали уровень сложности.

training_choice_users_count = events_df[events_df["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


In [548]:
# Изучаем покупку платных пакетов
# Следующий этап после выбора бесплатного набора вопросов — это возможная покупка платных пакетов. 
# Оценим число пользователей, которые совершили покупку. Для этого нам потребуется датафрейм purchase_df.

purchase_df['user_id'].nunique()

1600

In [549]:
# Рассчитаем процент пользователей percent_of_paying_users, которые оплатили вопросы,
# от числа пользователей, которые приобрели бесплатные вопросы:

paying_users_count = purchase_df["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


In [550]:
# Какой процент составляют пользователи, оплатившие пакеты вопросов (от числа зарегистрировавшихся пользователей)?
percent_of_paying_users = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа зарегистрировавшихся пользователей): {:.2%}".format(
        percent_of_paying_users
    )
)
# Какой вывод мы можем сделать?
# Итак, процент зарегистрировавшихся пользователей, которые приобрели платный пакет, составляет 8.03 %.
# Само по себе это число нам ни о чём не говорит, 
# поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп.
# И эта информация укажет нам возможные пути оптимизации.

Процент пользователей, которые оплатили вопросы (от числа зарегистрировавшихся пользователей): 8.03%


Анализ уникальных пользовательских путей

In [551]:
# Объединяем датафреймы
# Для начала давайте объединим датафрейм с событиями с датафреймом по оплатам. 
# Это позволит анализировать все эти события в рамках одной структуры данных
# Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase. 
# Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.
purchase_df['event_type'] = 'purchase'
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
 4   event_type      1600 non-null   object
dtypes: int64(3), object(2)
memory usage: 75.0+ KB


C:\Users\Yana\AppData\Local\Temp/ipykernel_11072/1446478013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_type'] = 'purchase'


In [552]:
# Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их несколько отличается, 
# так как столбец id в events_df указывает на идентификатор события, 
# а столбец id в purchase_df указывает на идентификатор оплаты. 
# Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах.

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [553]:
# Следующим шагом объединим датафреймы events_df и purchase_df с помощью функции pd.concat() 
# и запишем объединённый датафрейм в переменную total_events_df.

total_events_df = pd.concat([events_df,purchase_df],sort=False)

# Посмотрим, что получилось:

total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01T03:48:40,NaN,27832,NaN,NaN,NaN
51406,80309.0,registration,NaN,2018-01-01T04:07:25,NaN,27833,NaN,NaN,NaN
51407,80310.0,registration,NaN,2018-01-01T08:35:10,NaN,27834,NaN,NaN,NaN
51408,80311.0,registration,NaN,2018-01-01T11:54:47,NaN,27835,NaN,NaN,NaN
51409,80312.0,registration,NaN,2018-01-01T13:28:07,NaN,27836,NaN,NaN,NaN
51410,80313.0,registration,NaN,2018-01-01T14:08:40,NaN,27837,NaN,NaN,NaN
51411,80314.0,registration,NaN,2018-01-01T14:42:58,NaN,27838,NaN,NaN,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01T14:54:40,31505.0,27836,NaN,NaN,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01T15:00:51,31506.0,27835,NaN,NaN,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01T15:06:15,31506.0,27835,NaN,NaN,NaN


In [554]:
# Сколько получилось строк в объединённом датафрейме total_events? 68559 
#total_events_df.info()
total_events_df['event_type'].value_counts()

registration       19926
tutorial_start     18050
tutorial_finish    14904
level_choice        8342
pack_choice         5737
purchase            1600
Name: event_type, dtype: int64

In [555]:
# Теперь сбросим индексы объединённого датафрейма 
# (так как после объединения они дублировались и несут мало смысла) 
# с помощью метода reset_index() и отсортируем все события по возрастанию времени с помощью sort_values().

total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

In [556]:
# Теперь можно для каждого пользователя создать список, 
# который будет содержать во временной последовательности все события, совершаемые данным пользователем.
# Для этого воспользуемся методом groupby по столбцу event_type и применим агрегирующую функцию apply(list). 
# Таким образом мы сгруппируем строки по пользователю, 
# а затем объединим в списки содержимое столбца event_type по каждому пользователю. 
# Запишем результат в датафрейм user_path_df.

user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


In [557]:
# Теперь у нас есть датафрейм, где для каждого пользователя в столбце event_type 
# указаны последовательно все его события — пути пользователя.
# Если мы объединим данные по этой колонке и подсчитаем количество пользователей по каждому пути, 
# то получим наиболее популярные пути пользователей.

# Выявляем наиболее популярные пути

# Прежде чем мы сможем оценить популярные пути пользователей, преобразуем список событий в строку event_path. 
# Эта операция нужна для оптимизации скорости объединения, так как иначе Pandas может делать подсчёт слишком долго.

#user_path_df = (total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index())
#user_path_df.head(10)

In [558]:
# Теперь можно сгруппировать датафрейм по столбцу event_path, подсчитав число пользователей.

#user_paths = (user_path_df.groupby(["event_type"])["user_id"].nunique().sort_values(ascending=False))

# Выведем 10 самых популярных последовательностей.

user_paths.head(10)

# Сколько пользователей совершили наиболее популярный путь, ведущий к событию purchase? 1083

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

In [559]:
# Ищем последовательности с оплатой
# Давайте посмотрим, какие ещё последовательности содержат в себе оплату.

user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

Анализ временных промежутков

Определяем время между registration и tutorial_start

Сначала определим, какое время проходит между регистрацией и началом обучения.

Важно понимать, что это время мы можем определить только для таких пользователей, которые хотя бы раз начинали обучение.

In [560]:
# registration
# Первым делом выделим отдельный датафрейм registration_df, который будет содержать только события с event_type = registration. Этот датафрейм будет вспомогательным для определения времени между регистрацией и началом обучения.

registration_df = total_events_df[total_events_df['event_type'] == 'registration']

Проверим, что в нашем датафрейме registration_df для каждого пользователя содержится только по одному событию, то есть по одной регистрации на пользователя. Это важно, чтобы однозначно для одного пользователя присоединить только одного событие, по которому мы будем считать время.

Наличие более чем одной регистрации на пользователя может говорить о технической ошибке: например, какие-то события записываются в базу по несколько раз.

In [561]:
# Для определения среднего количества событий на пользователя мы можем воспользоваться методом value_counts(), 
# результатом которого будет серия с количеством событий на каждого пользователя.
# Затем, усреднив эти значения между собой, мы получим среднее число событий типа registration на одного пользователя.

registration_df['user_id'].value_counts().mean()

# Среднее число регистраций на одного пользователя равняется 1. Отлично!

1.0

In [562]:
# Оставим в датафрейме registration_df только те данные, которые нужны для наших вычислений 
# столбец user_id с идентификатором пользователя 
# и столбец start_time со временем регистрации. 
# Также переименуем столбец start_time в столбец registration_time для понятности.

registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

In [563]:
# tutorial_start  
# Теперь возьмёмся за следующее событие. 
# Выделим отдельный датафрейм tutorial_start_df, 
# который будет содержать только события с event_type = tutorial_start (начало обучения).

tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']

#Так же, как и в случае с регистрациями, посчитаем, сколько таких событий приходится на пользователя.

tutorial_start_df['user_id'].value_counts().mean()

1.522179119581717

А вот это уже интересно. Мы получили значение больше 1.

Почему так произошло?

Q: По какой причине на пользователя в среднем приходится больше одного начала обучения?
A: Один пользователь может начать более чем одно обучение за время пользования приложением

Верно: Если вы внимательно читали описание, то помните, что в нашем приложении пользователь при желании может проийти обучение несколько раз. Например, уже после оплаты пользователь может решить, что забыл, как пользоваться приложением, и снова пройти обучение. Или он может начать обучение, но не пройти его до конца сразу и забросить, а в следующий вход в приложение ему придётся начать обучение снова, если оно ему понадобится.

Мы выяснили причину, теперь нужно найти ответ на главный вопрос.

Что мы будем с этим сделать?

In [564]:
# У некоторых наших пользователей более чем по одному событию начала обучения. Оставим только первое обучение.
# Таким образом мы будем искать время между регистрацией и началом первого обучения пользователя.
# Создадим датафрейм tutorial_start_df_wo_duplicates, где будет присутствовать только первое обучение. 
# Для этого отсортируем датафрейм по start_time, чтобы сначала шли более ранние события начала обучения,
# а затем удалим дубликаты по столбцу user_id. 
# Таким образом, для каждого user_id останется только первое событие типа tutorial_start.

tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")

# Проверим, что для каждого пользователя осталось только одно событие.

tutorial_start_df_wo_duplicates['user_id'].value_counts().mean()

1.0

In [565]:
# Так же, как и в случае с датафреймом registration_df, оставим только такие столбцы, 
# которые пригодятся нам в дальнейшем. Это столбцы user_id, start_time, tutorial_id. 
# Также переименуем колонку start_time в tutorial_start_time:

tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})

 Объединяем датафреймы

Настало время объединить между собой данные двух получившихся датафреймов: registration_df и tutorial_start_df_wo_duplicates. Объединение позволит для каждого пользователя в рамках одного датафрейма указать и время регистрации, и время начала первого обучения в приложении.

Объединить датафреймы мы можем с помощью функции merge(), объединяя данные по параметру user_id. Также нам нужно объединить данные с использованием параметра how='inner', это позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, которые есть в обоих датафреймах. Вспомните SQL, это очень похоже на INNER JOIN.

In [566]:
merged_df = registration_df.merge(
    tutorial_start_df_wo_duplicates, on="user_id", how="inner"
)
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time
0,27833,2018-01-01T04:07:25,31508.0,2018-01-01T17:47:40
1,27834,2018-01-01T08:35:10,31510.0,2018-01-01T19:46:11
2,27835,2018-01-01T11:54:47,31506.0,2018-01-01T15:00:51
3,27836,2018-01-01T13:28:07,31505.0,2018-01-01T14:54:40
4,27839,2018-01-01T18:24:01,31509.0,2018-01-01T19:11:36


In [567]:
# Сделаем столбец timedelta, 
# в котором посчитаем разницу между временем начала обучения (tutorial_start_time) 
# и временем регистрации (registration_time):

merged_df['registration_time'] = pd.to_datetime(merged_df['registration_time'])
merged_df['tutorial_start_time'] = pd.to_datetime(merged_df['tutorial_start_time'])
merged_df.info()

merged_df["timedelta"] = (
    merged_df["tutorial_start_time"] - merged_df["registration_time"]
)
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11858 entries, 0 to 11857
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              11858 non-null  int64         
 1   registration_time    11858 non-null  datetime64[ns]
 2   tutorial_id          11858 non-null  float64       
 3   tutorial_start_time  11858 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 463.2 KB


,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


In [568]:
# Усреднив с помощью функции mean() значения в столбце timedelta, 
# мы получим среднее время, которое проходит между регистрацией пользователя и началом первого обучения:

merged_df["timedelta"].mean()

Timedelta('0 days 04:38:24.019817844')

In [569]:
# Как видим, между регистрацией и началом обучения в среднем проходит 4 часа 38 минут.
# Но это в среднем. Среднее бывает коварно, особенно если у нас есть несколько пользователей, 
# которые слишком долго проходят обучение. Давайте воспользуемся методом describe(), 
# чтобы понять некоторые важные параметры.

merged_df['timedelta'].describe()

# К чему мы пришли?
# В одном из прошлых кейсов мы говорили о квартилях. 
# Здесь мы их явно видим: первый квартиль составляет 1 час 21 минут. 
# А медиана (второй квартиль) — 3 часа 22 минут.

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

Такая сегментация пользователей может дать нам важные опорные точки для оптимизации времени. Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время.

Также мы можем отдельно выделить тех пользователей, у которых время больше медианного, чтобы исследовать их детально и понять, что мешает им быстрее дойти до этапа обучения.

Определяем время между registration и level_choice

In [570]:
# По такому же алгоритму рассчитаем время между выбором уровня сложности и регистрацией:
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
total_events_df['event_datetime'] = pd.to_datetime(total_events_df['event_datetime'])

total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 0 to 68558
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


Определяем время между level_choice и Pack_choice

In [571]:
# Сколько в среднем для всех пользователей проходит времени между 
# событием выбора уровня сложности тренировки до события выбора набора бесплатных вопросов?
pack_choise_df = total_events_df[total_events_df["event_type"] == "pack_choice"]
print(pack_choise_df["user_id"].value_counts().mean()) #should be 1.0
pack_choise_df = pack_choise_df[["user_id", "start_time"]].rename(
    columns={"start_time": "pack_choice_time"})

merged_df_4 = level_choice_df.merge(pack_choise_df, on="user_id", how="inner")

merged_df_4["timedelta"] = (merged_df_4["pack_choice_time"] - merged_df_4["level_choice_time"])

print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

1.0
0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


Определяем время между Pack_choice и purchase

In [572]:
# Сколько в среднем для всех пользователей проходит времени между 
# событием выбора бесплатных вопросов и первой оплатой?

total_events_df["event_type"].value_counts()
purchase_df_2 = total_events_df[total_events_df["event_type"] == "purchase"]
print(purchase_df_2["user_id"].value_counts().mean()) #should be 1.0
purchase_df_2 = purchase_df_2[["user_id", "event_datetime"]].rename(columns={"event_datetime": "purchase_time"})
# purchase_df_2.head()

merged_df_5 = purchase_df_2.merge(pack_choise_df, on="user_id", how="inner")

merged_df_5["timedelta"] = (merged_df_5["purchase_time"] - merged_df_5["pack_choice_time"])
#merged_df_5.head()
print(merged_df_5["timedelta"].mean())
print(merged_df_5["timedelta"].describe())

1.0
3 days 17:46:53.403125
count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object


Проверка аналитической гипотезы

Ищем пользователей, прошедших обучение хотя бы раз

In [573]:
# Давайте разобьём наших пользователей на группы.
# Сначала определим пользователей, которые прошли обучение хотя бы раз. 
# Запишем идентификаторы этих пользователей в переменную users_with_finished_tutorial.
# Тут всё просто, это пользователи, у которых есть событие tutorial_finish: 10250

users_with_finished_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_finish"
]["user_id"].unique()
print(len(users_with_finished_tutorial))

10250


Ищем пользователей, которые начали, но не закончили обучение

In [574]:
# Затем найдём пользователей, которые начали обучение, но не прошли его ни разу. 1608
# Это будут пользователи, у которых есть событие tutorial_start, но нет события tutorial_finish.

# Проще всего сначала найти пользователей, у которых есть событие tutorial_start, 
# а после убрать из списка таких пользователей, у которых есть событие tutorial_finish.

# В этом нам поможет работа со множествами:

users_with_started_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_start"
]["user_id"].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (
    set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
)

print(len(set_users_with_started_tutorial))
print(len(set_users_not_finished_but_started_tutorial))
print(
    len(set_users_with_started_tutorial) - len(set(users_with_finished_tutorial))
    == len(set_users_not_finished_but_started_tutorial)
)

11858
1608
True


Ищем пользователей, сразу выбравших уровень сложности

In [575]:
# И последняя группа пользователей — те, кто ни разу не проходил обучение. 8068
# У таких пользователей отсутствует событие tutorial_start. 
# Поэтому мы можем просто взять и убрать из множества всех пользователей множество set_users_with_started_tutorial:

all_users = total_events_df["user_id"].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(
    set_users_with_started_tutorial
)
print(len(set_users_not_started_tutorial))
print(
    len(set_all_users) - len(set_users_with_started_tutorial)
    == len(set_users_not_started_tutorial)
)

8068
True


In [576]:
# Проверим, что мы правильно разбили пользователей на группы.
# Просуммировав длину всех множеств пользователей, мы должны получить исходное количество всех пользователей:

len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial) + len(users_with_finished_tutorial) == len(set_all_users)

True

Считаем пользователей, которые завершили обучение и совершили оплату

In [577]:
# Сначала сделаем датафрейм purchase_df_1, в котором будут данные по оплатам пользователей, которые завершили обучение.

purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)]

# Затем посчитаем, сколько пользователей в этом датафрейме. 1447

purchase_df_1['user_id'].nunique()

1447

In [582]:
# Поделив количество пользователей в датафрейме purchase_df_1 на общее количество пользователей,
# которые завершили обучение, мы получим процент оплативших пользователей в этой группе. 14,12%

percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(
    users_with_finished_tutorial
)
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, завершивших обучение): 14.12%


Считаем средний размер платежа разных групп

In [579]:
# Дальше найдём средний размер платежа для этих пользователей.

purchase_df_1['amount'].mean()

# Таким образом, средний размер платежа у пользователей, 
# которые завершили обучение, составляет 110 рублей 99 копейки.

110.98825155494126

In [581]:
# Давайте сделаем датафрейм purchase_df_2, в котором будут оплаты пользователей, начавших, но не закончивших обучение.
# И найдём, какой процент таких пользователей оплачивает пакеты вопросов, от общего числа пользователей:

purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

#Итак, мы видим, что процент меньше, чем среди тех, кто закончил обучение. 8,15%

131
Процент пользователей, которые оплатили вопросы (от числа пользователей, начавших обучение, но не завершивших): 8.15%


In [583]:
#А что у нас со средним чеком? 104,96

purchase_df_2['amount'].mean()

# Средний чек тоже меньше. Скорее всего, между прохождением обучения и оплатой есть зависимость.

104.9618320610687

Предлагаем вам самостоятельно определить, как обстоят дела у последней группы — пользователей, которые ни разу не начинали обучение (set_users_not_started_tutorial).

In [586]:
# # Давайте сделаем датафрейм purchase_df_3, в котором будут оплаты пользователей, не начинавших обучение.
# И найдём, какой процент таких пользователей оплачивает пакеты вопросов, от общего числа пользователей:

purchase_df_3 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_started_tutorial)
]
print(purchase_df_3["user_id"].nunique())
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(
    set_users_not_started_tutorial
)
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, не начавших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

22
Процент пользователей, которые оплатили вопросы (от числа пользователей, не начавших обучение): 0.27%


In [588]:
purchase_df_3['amount'].mean() # А что у нас со средним чеком?

128.4090909090909

In [593]:
total_events_df.to_csv (r'C:\Users\Yana\Documents\total_events.csv', index = False, header=True)